In [ ]:
import os
import re
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import colormaps as cm

import folium
from folium import plugins
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point, LineString, box
import networkx as nx
import gurobipy as gb

# Project setup

## Load data

In [ ]:
# Get segment and stops on route
def get_segment_stops(
    segment_df: pd.DataFrame,
    stops_df: pd.DataFrame,
    route_id: str,
    direction_id: int = 0,
):
    segment_distance = segment_df.query(
        "route_id == @route_id & direction_id == @direction_id"
    )

    stops_in_route = stops_df[
        stops_df["stop_id"].isin(
            list(
                set(
                    segment_distance[["start_stop_id", "end_stop_id"]].values.reshape(
                        -1
                    )
                )
            )
        )
    ]

    return segment_distance, stops_in_route


# Get unique stops on route
def get_stops_on_route(
    route_ids: list, segment_df: pd.DataFrame, stops_df: pd.DataFrame
):
    route_details = {}

    for r in route_ids:
        detail = get_segment_stops(segment_df, stops_df, r, 0)

        route_details[r] = {}
        route_details[r]["segment"] = detail[0]
        route_details[r]["stops"] = detail[1]

    stops_df = pd.concat(
        [v["stops"] for k, v in route_details.items()]
    ).drop_duplicates()

    print(f"Number of total stops in routes: {len(stops_df)}")

    stops_df["geometry"] = stops_df.apply(
        lambda x: Point((float(x.stop_lon), float(x.stop_lat))), axis=1
    )

    stops_df_gpd = gpd.GeoDataFrame(
        stops_df.drop(
            columns=["location_type", "parent_station", "wheelchair_boarding"]
        ),
        geometry="geometry",
    )

    display(stops_df_gpd.head())

    return stops_df_gpd

In [ ]:
segments = pd.read_csv("segments.csv")
stops = pd.read_csv("stops.csv")

segments[["route_id", "start_stop_id", "end_stop_id"]] = segments[
    ["route_id", "start_stop_id", "end_stop_id"]
].astype(str)

routes = ["24", "51", "67", "18", "33", "45", "80"]

stops_df_gpd = get_stops_on_route(routes, segments, stops)

#### Add depot

In [ ]:
def add_depot(lat: float, lon: float, stops_df: pd.DataFrame):
    depot = pd.DataFrame(
        {
            "stop_id": ["0"],
            "stop_name": ["Depot"],
            "stop_lat": [lat],
            "stop_lon": [lon],
            "stop_code": 0.0,
        }
    )

    depot["geometry"] = Point((float(depot.stop_lon), float(depot.stop_lat)))

    stops_df_gpd = pd.concat([depot, stops_df]).reset_index(drop=True)

    return depot, stops_df_gpd

### Take a random sample of stops

In [ ]:
random_stops_df_gpd = stops_df_gpd.sample(n=20, random_state=5)
depot, random_stops_df_gpd = add_depot(45.509642, -73.580091, random_stops_df_gpd)

print(f"Number of stops in sample: {len(random_stops_df_gpd)}")

display(random_stops_df_gpd.head())

## Calculate distance matrix

In [ ]:
# G = ox.graph_from_place("Montreal, Canada", network_type="drive")
G = ox.load_graphml("montreal_drive.graphml")

In [ ]:
def get_distance_matrix(stops_df: pd.DataFrame, G: nx.Graph):
    if os.path.exists("distance_matrix.json"):
        print("Loading distance matrix from JSON")
        distance_matrix = pd.read_json("distance_matrix.json")

        distance_matrix.columns = distance_matrix.columns.astype(str)
        distance_matrix.index = distance_matrix.index.astype(str)

        if len(distance_matrix) != len(stops_df):
            print("⚠️ Distance matrix does not match number of stops")
            print(f"Number of stops in distance matrix: {len(distance_matrix)}")
            print(f"Number of stops in DataFrame: {len(stops_df)}")
        else:
            print("✅ Distance matrix loaded successfully")
            print(f"Number of stops in distance matrix: {len(distance_matrix)}")
            print(f"Number of stops in DataFrame: {len(stops_df)}")

        return distance_matrix

    print(f"Calculating distance for {len(stops_df)} stops")

    distance_matrix = np.zeros((len(stops_df), len(stops_df)))

    for i, stop1 in enumerate(stops_df.itertuples()):
        print(f"Calculating distance for stop {i}")
        for j in range(i + 1, len(stops_df)):
            stop2 = stops_df.iloc[j]

            origin = ox.nearest_nodes(G, stop1.stop_lon, stop1.stop_lat)
            destination = ox.nearest_nodes(G, stop2.stop_lon, stop2.stop_lat)

            try:
                distance = nx.shortest_path_length(
                    G, origin, destination, weight="length"
                )
            except nx.NetworkXNoPath:
                distance = np.Inf

            distance_matrix[i, j] = distance
            distance_matrix[j, i] = distance

        print("-" * 50)

    # Convert to km and save as JSON
    distance_matrix = pd.DataFrame(
        distance_matrix / 1000,
        columns=stops_df.stop_id,
        index=stops_df.stop_id,
    )

    distance_matrix.to_json("distance_matrix.json")

    return distance_matrix

In [ ]:
distance_matrix = get_distance_matrix(random_stops_df_gpd, G)

## Add disaster area

In [ ]:
def add_disaster_area(stops_df: pd.DataFrame, disaster_bounds: list):
    disaster_area = box(*disaster_bounds)

    stops_df_gpd = gpd.GeoDataFrame(stops_df, geometry="geometry")

    stops_in_disaster_area = stops_df_gpd[stops_df_gpd.within(disaster_area)]

    print(f"Number of stops in disaster area: {len(stops_in_disaster_area)}")

    display(stops_in_disaster_area)

    return stops_in_disaster_area, disaster_area


stops_in_disaster_area, disaster_area = add_disaster_area(
    random_stops_df_gpd, [-73.666166, 45.547247, -73.591552, 45.637577]
)

## View stops in sample

In [ ]:
stops_map = folium.Map(
    location=[45.5048542, -73.5691235],
    zoom_start=11,
    tiles="cartodbpositron",
    width="100%",
)

# Add disaster area
folium.GeoJson(
    disaster_area,
    name="Disaster area",
    style_function=lambda x: {
        "color": "#ff0000",
        "fillColor": "#ff0000",
        "weight": 1,
        "fillOpacity": 0.4,
    },
).add_to(stops_map)

# Add stops in network
for stop in random_stops_df_gpd.itertuples():
    folium.CircleMarker(
        location=[stop.stop_lat, stop.stop_lon],
        radius=5,
        color=(
            "red"
            if stop.stop_id in stops_in_disaster_area.stop_id.values.tolist()
            else "darkgreen"
        ),
        fill=True,
        fill_opacity=1,
        fill_color=(
            "red"
            if stop.stop_id in stops_in_disaster_area.stop_id.values.tolist()
            else "darkgreen"
        ),
        tooltip=f"{stop.stop_name} ({stop.stop_id})",
        popup=f"""
        <div>
            <h4>{stop.stop_name} ({stop.stop_id})</h4>
            <h4>Distance from depot: {distance_matrix.loc["0", stop.stop_id]:.1f} km</h4>
        </div>
        """,
    ).add_to(stops_map)

folium.plugins.Fullscreen(position="topright").add_to(stops_map)
folium.plugins.MousePosition(position="topright").add_to(stops_map)

stops_map

# MILP Model - Split delivery vehicle routing problem

## Parameters

In [ ]:
# rng = np.random.default_rng(5)

# num_buses = 10
# BUS_CAPACITY = 75

# distance_matrix_model = distance_matrix.drop(
#     columns=stops_in_disaster_area.stop_id, index=stops_in_disaster_area.stop_id
# )

# stops = list(distance_matrix_model.columns)
# num_stops = len(stops)

# distance_matrix_model = distance_matrix_model.loc[stops, stops]

# demand = {stop: rng.integers(0, 100) for stop in stops}
# demand[stops[0]] = 0


# print(f"Total demand: {sum(demand.values())}")
# print(f"Total capacity: {num_buses * BUS_CAPACITY}")

## Split nodes

### Split methods

#### Geometric progression

$$
\begin{align*}
D_{ix} &= \frac{2^{x-1}}{\sum_{i=1}^{S} 2^{x-1}} D_{i}
\end{align*}
$$

$D_{ix}$ is rounded down to the nearest integer. If $\sum_{i=1}^{S} D_{ix} < D_{i}$, then $D_{iS} = D_{iS} + D_{i} - \sum_{i=1}^{S} D_{ix}$.

In [ ]:
# Split the node demand into smaller demands according to a geometric progression
def split_demand_geometric(node_demand, BUS_CAPACITY, fraction, fraction_sum):
    demands = [np.floor(node_demand * (f / fraction_sum)) for f in fraction]
    demands = list(filter(lambda x: x > 0, demands))

    if sum(demands) < node_demand:
        demands[-1] += node_demand - sum(demands)

    i = len(demands) - 1

    while demands[i] >= BUS_CAPACITY * 1:
        new_demand = split_demand_geometric(demands[i])
        demands.pop(i)
        demands.extend(new_demand)
        i -= 1

    return demands

#### Capacity-based split

$$
\begin{align*}
D_{ix} &= Q &\quad \forall x \in \{1, \dots, S = \frac{q_{i}}{Q}\} \\
\end{align*}
$$

$D_{ix}$ is rounded down to the nearest integer. If $\sum_{i=1}^{S} D_{ix} < D_{i}$, then $D_{i(S+1)} = D_{i} - \sum_{i=1}^{S} D_{ix}$.


In [ ]:
# Split the node demand into smaller demands according to BUS_CAPACITY
def split_demand_capacity(node_demand, BUS_CAPACITY):
    demands = [BUS_CAPACITY for _ in range(int(np.floor(node_demand / BUS_CAPACITY)))]
    demands.append(node_demand - sum(demands))

    return demands

#### Equal split

$$
\begin{align*}
D_{ix} &= 1 &\quad \forall x \in \{1, \dots, q_{i}\} \\
\end{align*}
$$


In [ ]:
# Split the node demand into equal demand nodes of 1
def split_demand_equal(node_demand, BUS_CAPACITY):
    demands = [1 for _ in range(node_demand)]

    return demands

#### Random split

$$
\begin{align*}
D_{ix} &= Z \sim \{1, Q\}
\end{align*}
$$


In [ ]:
# Split the node demand into random demand values

def split_demand_random(node_demand, BUS_CAPACITY, rng: np.random.Generator):
    demands = []
    remaining_demand = node_demand

    while remaining_demand > 0:
        demand = rng.integers(1, BUS_CAPACITY)
        demands.append(demand)
        remaining_demand -= demand

    if sum(demands) > node_demand:
        demands[-1] -= sum(demands) - node_demand

    return demands

#### Split individual demand nodes

In [ ]:
def split_demand_node(demand, node, BUS_CAPACITY, rng, split_type: str = "geometric"):
    node_demand = demand[node]
    match split_type:
        case "geometric":
            S = 100
            fraction = [2 ** (i - 1) for i in range(1, S + 1)]
            fraction_sum = sum(fraction)
            demands = split_demand_geometric(
                node_demand, BUS_CAPACITY, fraction, fraction_sum
            )
        case "capacity":
            demands = split_demand_capacity(node_demand, BUS_CAPACITY)
        case "equal":
            demands = split_demand_equal(node_demand, BUS_CAPACITY)
        case "random":
            demands = split_demand_random(node_demand, BUS_CAPACITY, rng)
        case _:
            raise ValueError("Invalid demand split type")

    return {f"{node}_{i}": d for i, d in enumerate(demands, 1)}

### Reconstruct parameters

In [ ]:
def reconstruct_demand(demand, BUS_CAPACITY, rng, split_type: str = "geometric"):
    nodes_exceeding_demand = {}
    new_demand = demand.copy()
    for k, v in demand.items():
        if v > BUS_CAPACITY:
            new_nodes = split_demand_node(demand, k, BUS_CAPACITY, rng, split_type)

            new_demand.pop(k)
            new_demand.update(new_nodes)

            nodes_exceeding_demand[k] = new_nodes

    print(f"New nodes added for: {list(nodes_exceeding_demand.keys())}")

    return new_demand, nodes_exceeding_demand

In [ ]:
# Update distance matrix to include split nodes. Distance between split nodes of same parent node is 0
def update_distance_matrix(distance_matrix, nodes_exceeding_demand, new_demand):
    distance_matrix_model = distance_matrix.copy()

    for node in nodes_exceeding_demand:
        for i in range(1, len([k for k in new_demand.keys() if node in k]) + 1):
            distance_matrix_model[f"{node}_{i}"] = distance_matrix_model[node]
            distance_matrix_model.loc[f"{node}_{i}"] = distance_matrix_model.loc[node]
            distance_matrix_model.loc[f"{node}_{i}", node] = 0

        distance_matrix_model.drop(columns=[node], index=[node], inplace=True)

    return distance_matrix_model

In [ ]:
# new_demand, nodes_exceeding_demand = reconstruct_demand(demand, BUS_CAPACITY, "geometric")
# distance_matrix_model_pruned = update_distance_matrix(
#     distance_matrix_model, nodes_exceeding_demand, new_demand
# )

In [ ]:
# for k, v in new_demand.items():
#     assert v <= BUS_CAPACITY

# assert sum(new_demand.values()) == sum(demand.values())

In [ ]:
# new_stops = list(distance_matrix_model_pruned.columns)
# num_new_stops = len(new_stops)

# print(f"Total number of stops (overall): {num_stops}")
# print(f"Total number of stops (including splits): {num_new_stops}")

## Model

In [ ]:
def define_model(
    stops: list,
    buses: int,
    demand: dict,
    distance_matrix: pd.DataFrame,
    BUS_CAPACITY: int,
    DISTANCE_THRESHOLD: float = 5.0,
    **params,
):
    # ----------------------------------------------------------------------------------------------
    # Model

    model = gb.Model("Bus Routing")
    model.Params.MIPGap = params.get("MIPGap", 0.05)
    model.Params.TimeLimit = params.get("TimeLimit", 60 * 3)
    model.Params.MIPFocus = params.get("MIPFocus", 1)
    model.Params.LogToConsole = params.get("LogToConsole", 1)

    # ----------------------------------------------------------------------------------------------
    # Decision Variables

    x = model.addVars(
        stops,
        stops,
        buses,
        vtype=gb.GRB.BINARY,
        name=(
            f"{i} -> {j} (bus {k})" for i in stops for j in stops for k in range(buses)
        ),
    )

    u = model.addVars(
        stops,
        vtype=gb.GRB.INTEGER,
        name=(f"Load at Stop {i}" for i in stops),
    )

    # ----------------------------------------------------------------------------------------------
    # Objective Function
    model.setObjective(
        gb.quicksum(
            distance_matrix.loc[i, j] * x[i, j, k]
            for i in stops
            for j in stops
            for k in range(buses)
        ),
        gb.GRB.MINIMIZE,
    )

    # ----------------------------------------------------------------------------------------------
    # Constraints

    # Vehicle leaves nodes that it enters
    model.addConstrs(
        (
            gb.quicksum(x[j, i, k] for j in stops)
            == gb.quicksum(x[i, j, k] for j in stops)
            for i in stops
            for k in range(buses)
        ),
        name="Vehicle leaves nodes that it enters",
    )

    # Every node is entered once
    model.addConstrs(
        (
            gb.quicksum(x[i, j, k] for i in stops for k in range(buses)) == 1
            for j in stops[1:]
        ),
        name="Every node is entered once",
    )

    # Every vehicle leaves the depot
    model.addConstrs(
        (gb.quicksum(x[stops[0], j, k] for j in stops[1:]) <= 1 for k in range(buses)),
        name="Every vehicle may leave the depot if needed",
    )

    # Capacity constraint
    model.addConstrs(
        (
            gb.quicksum(demand[j] * x[i, j, k] for j in stops[1:] for i in stops)
            <= BUS_CAPACITY
            for k in range(buses)
        ),
        name="Capacity constraint",
    )

    # No travel between same node
    model.addConstrs(
        (x[i, i, k] == 0 for i in stops for k in range(buses)),
        name="No same node",
    )

    # Subtour elimination constraints
    model.addConstrs(
        (
            u[j] - u[i] >= demand[j] - BUS_CAPACITY * (1 - x[i, j, k])
            for i in stops[1:]
            for j in stops[1:]
            for k in range(buses)
            if i != j
        ),
        name="Subtour elimination constraint",
    )

    model.addConstrs(
        (u[i] >= demand[i] for i in stops[1:]),
        name="Lower bound for u",
    )

    model.addConstrs(
        (u[i] <= BUS_CAPACITY for i in stops[1:]),
        name="Upper bound for u",
    )

    # Distance between two travel nodes is less than specified distance
    model.addConstrs(
        (
            distance_matrix.loc[i, j] * x[i, j, k] <= DISTANCE_THRESHOLD
            for i in stops[1:]
            for j in stops[1:]
            for k in range(buses)
        ),
        name="Distance between two travel nodes is less than a specified distance",
    )

    # ----------------------------------------------------------------------------------------------
    # Solve model
    model._vars = x
    model.update()

    return model

In [ ]:
# model = define_model(
#     new_stops,
#     num_buses,
#     new_demand,
#     distance_matrix_model_pruned,
#     BUS_CAPACITY,
#     MIPGap=0.2,
#     TimeLimit=60 * 1,
#     MIPFocus=1,
# )

# model.optimize()

## Solution

### Optimal routes

In [ ]:
def show_solution(model, stops, buses, demand, nodes_exceeding_demand, distance_matrix, stops_df):
    x = model._vars

    print('-'*100)
    print(f"Objective value: {model.objVal:.2f} km")
    print('-'*100)

    # Distance by bus
    distance_bus = pd.DataFrame(
        {
            "bus": k,
            "distance": sum(
                distance_matrix.loc[i, j] * x[i, j, k].x for i in stops for j in stops
            ),
        }
        for k in range(buses)
    )

    distance_bus = distance_bus[distance_bus["distance"] > 0].reset_index(drop=True)

    # display(distance_bus)

    # Bus paths
    bus_path = {}

    for k in range(buses):
        bus_path[k] = []
        for i in stops:
            for j in stops:
                if x[i, j, k].x == 1:
                    bus_path[k].append(
                        {
                            "start_stop": i,
                            "end_stop": j,
                            "distance": distance_matrix.loc[i, j],
                        }
                    )

    # Convert to dataframe with bus route number
    bus_path_df = []

    for k, v in bus_path.items():
        bus_path_df.append(pd.DataFrame(v).assign(bus=k))

    bus_path_df = pd.concat(bus_path_df)

    paths = {}

    grouped = bus_path_df.groupby("bus")

    # Iterate over each bus group
    for bus, group in grouped:
        sorted_group = group.sort_values(by=["start_stop", "end_stop"]).reset_index(
            drop=True
        )
        path = ["0"]  # Initialize the path with the depot

        # Start with the first stop after the depot
        current_stop = sorted_group.loc[
            sorted_group["start_stop"] == "0", "end_stop"
        ].values[0]
        path.append(current_stop)

        # Follow the chain of stops
        while True:
            # Find the next stop where the current stop is the start stop
            next_stop = sorted_group.loc[
                sorted_group["start_stop"] == current_stop, "end_stop"
            ].values
            if not next_stop:
                break  # If there is no next stop, we've completed the path
            next_stop = next_stop[0]

            # Add the next stop to the path and set it as the current stop
            if next_stop == "0":
                break  # If the next stop is the depot, we've completed the path
            path.append(next_stop)
            current_stop = next_stop

        # Store the path for this bus
        paths[bus] = path

    bus_path_df["step"] = bus_path_df.apply(
        lambda x: paths[x.bus].index(x.start_stop), axis=1
    )

    bus_path_df.sort_values(by=["bus", "step"], inplace=True)

    bus_path_df["demand"] = bus_path_df["end_stop"].map(demand)

    bus_path_df[["start_stop", "end_stop"]] = bus_path_df[
        ["start_stop", "end_stop"]
    ].applymap(lambda x: x.split("_")[0])

    bus_path_df = bus_path_df.merge(
        stops_df[["stop_id", "stop_name", "geometry"]],
        left_on="end_stop",
        right_on="stop_id",
        how="left",
    )

    bus_path_df["step_demand"] = bus_path_df.groupby("bus")["demand"].cumsum()

    bus_path_df["is_split"] = bus_path_df["stop_id"].isin(nodes_exceeding_demand.keys())

    num_buses_used = bus_path_df.bus.nunique()
    print(f"Number of buses used: {num_buses_used}")

    # display(bus_path_df)

    # Print routes
    # for route in paths:
    #     print(f"Bus {route}: {' -> '.join(paths[route])} -> 0")

    return distance_bus, bus_path_df, paths

In [ ]:
# distance_bus, bus_path_df, paths = show_solution(
#     model,
#     new_stops,
#     num_buses,
#     new_demand,
#     distance_matrix_model_pruned,
#     random_stops_df_gpd,
# )

In [ ]:
def plot_networkx_bus(bus_path_df):
    plots = {}
    for bus in bus_path_df.bus.unique():
        # print(f"Bus {bus + 1}:")

        bus_route = bus_path_df[bus_path_df["bus"] == bus].reset_index(drop=True)

        fig = plt.figure(figsize=(20, 10))

        g = nx.DiGraph()

        for segment in bus_route.itertuples():
            g.add_edge(
                segment.start_stop,
                segment.end_stop,
                weight=segment.distance,
                step=segment.step,
                demands={"demand": segment.demand, "load": segment.step_demand},
                label=f"{segment.start_stop}-{segment.end_stop}",
            )

        pos = nx.circular_layout(g)

        nx.draw_networkx(g, pos, with_labels=True, node_size=500, node_color="skyblue")
        nx.draw_networkx_edge_labels(
            g, pos, edge_labels=nx.get_edge_attributes(g, "demands")
        )

        plots[bus] = fig

    plt.close("all")
    return plots


# network_plots = plot_networkx_bus(bus_path_df)

In [ ]:
# for k, v in network_plots.items():
#     print(f"Bus {k}:")
#     display(v)
#     print("-" * 100)

In [ ]:
def build_route_df(bus_path_df, depot):
    routes_gdf = (
        gpd.GeoDataFrame(bus_path_df.groupby("bus")["geometry"].apply(list))
        .rename(columns={"points": "geometry"})
        .reset_index()
    )

    # add depot to each geometry
    routes_gdf["points"] = routes_gdf.apply(
        lambda x: [Point((float(depot.stop_lon), float(depot.stop_lat)))] + x.geometry,
        axis=1,
    )

    routes_gdf["geometry"] = routes_gdf["points"].apply(LineString)
    routes_gdf.drop(columns=["points"], inplace=True)

    routes_gdf = routes_gdf.merge(
        bus_path_df.groupby("bus")["demand"].sum().rename("demand").reset_index(),
        on="bus",
    )

    routes_gdf = gpd.GeoDataFrame(routes_gdf, geometry="geometry")
    routes_gdf.crs = "EPSG:4326"

    # display(routes_gdf)

    return routes_gdf

In [ ]:
# routes_gdf = build_route_df(bus_path_df, depot)

### Map the routes

In [ ]:
def plot_routes(distance_matrix, bus_path_df, routes_gdf, num_buses, split_type):
    colormap_route = [
        mcolors.rgb2hex(c) for c in list(plt.cm.rainbow(np.linspace(0, 1, num_buses)))
    ]

    route_map = folium.Map(
        location=[45.5048542, -73.5691235],
        zoom_start=11,
        tiles="cartodbpositron",
        width="100%",
    )

    folium.GeoJson(
        disaster_area,
        name="Disaster area",
        style_function=lambda x: {
            "color": "#ff0000",
            "fillColor": "#ff0000",
            "weight": 1,
            "fillOpacity": 0.3,
        },
    ).add_to(route_map)

    for stop in bus_path_df.itertuples():
        folium.CircleMarker(
            location=[stop.geometry.coords[0][1], stop.geometry.coords[0][0]],
            radius=5,
            color=colormap_route[stop.bus] if stop.stop_id != "0" else "black",
            fill=True,
            fill_opacity=1,
            fill_color=colormap_route[stop.bus] if stop.stop_id != "0" else "black",
            tooltip=f"""
            <b>{stop.stop_name} ({stop.stop_id})</b>
            <br>
            Route: {bus_path_df[bus_path_df["stop_id"] == stop.stop_id]['bus'].values[0] + 1}
            <br>
            Step: {bus_path_df[bus_path_df["stop_id"] == stop.stop_id]['step'].values[0] + 1}
            <br>
            Demand: {bus_path_df[bus_path_df["stop_id"] == stop.stop_id]['demand'].values[0]}
            <br>
            Load: {bus_path_df[bus_path_df["stop_id"] == stop.stop_id]['step_demand'].values[0]}
            """,
            popup=f"""
            <div>
                <h4>{stop.stop_name} ({stop.stop_id})</h4>
                <h4>Distance from depot: {distance_matrix.loc["0", stop.stop_id]:.1f} km</h4>
            </div>
            """,
        ).add_to(route_map)

    for stop in stops_in_disaster_area.itertuples():
        folium.CircleMarker(
            location=[stop.geometry.coords[0][1], stop.geometry.coords[0][0]],
            radius=5,
            color="red",
            fill=True,
            fill_opacity=1,
            fill_color="red",
            tooltip=f"""
            <b>{stop.stop_name} ({stop.stop_id})</b>
            """,
            popup=f"""
            <div>
                <h4>{stop.stop_name} ({stop.stop_id})</h4>
                <h4>Distance from depot: {distance_matrix.loc["0", stop.stop_id]:.1f} km</h4>
            </div>
            """,
        ).add_to(route_map)

    for route in routes_gdf.itertuples():
        route_layer = folium.FeatureGroup(f"Route {route.bus + 1}")
        folium.PolyLine(
            locations=[(p[1], p[0]) for p in route.geometry.coords],
            color=colormap_route[route.bus],
            weight=3,
            opacity=0.6,
            tooltip=f"Route {route.bus + 1}",
            popup=f"""
            <div>
                <h5>Route {route.bus + 1}</h5>
                <h5>Total demand: {route.demand}</h5>
            </div>
            """,
        ).add_to(route_layer)

        route_layer.add_to(route_map)

    folium.plugins.Fullscreen(position="topright").add_to(route_map)
    folium.plugins.MousePosition(position="topright").add_to(route_map)
    folium.LayerControl().add_to(route_map)

    route_map.save(f"route_map_split_{split_type}.html")
    return route_map

# MILP model as a function

In [ ]:
def solve_model(
    depot,
    stops_df,
    distance_matrix,
    num_buses,
    BUS_CAPACITY,
    DEMAND_LIMIT: int = 100,
    DISTANCE_THRESHOLD: float = 5.0,
    split_type: str = "geometric",
    MIPGap: float = 0.2,
    TimeLimit: int = 60 * 3,
    MIPFocus: int = 1,
    LogToConsole: int = 1,
):
    rng = np.random.default_rng(5)

    distance_matrix_model = distance_matrix.drop(
        columns=stops_in_disaster_area.stop_id, index=stops_in_disaster_area.stop_id
    )

    stops = list(distance_matrix_model.columns)
    num_stops = len(stops)

    distance_matrix_model = distance_matrix_model.loc[stops, stops]

    demand = {stop: rng.integers(1, DEMAND_LIMIT) for stop in stops}
    demand[stops[0]] = 0

    print('-'*100)
    print(f"Total demand: {sum(demand.values())}")
    print(f"Total capacity: {num_buses * BUS_CAPACITY}")
    print('-'*100)

    new_demand, nodes_exceeding_demand = reconstruct_demand(
        demand, BUS_CAPACITY, rng, split_type
    )

    distance_matrix_model_pruned = update_distance_matrix(
        distance_matrix_model, nodes_exceeding_demand, new_demand
    )

    for k, v in new_demand.items():
        assert v <= BUS_CAPACITY

    assert sum(new_demand.values()) == sum(demand.values())

    new_stops = list(distance_matrix_model_pruned.columns)
    num_new_stops = len(new_stops)

    print('-'*100)
    print(f"Total number of stops: {num_stops}")
    print(f"Total number of new stops (including splits): {num_new_stops}")
    print('-'*100)

    model = define_model(
        new_stops,
        num_buses,
        new_demand,
        distance_matrix_model_pruned,
        BUS_CAPACITY,
        DISTANCE_THRESHOLD=DISTANCE_THRESHOLD,
        MIPGap=MIPGap,
        TimeLimit=TimeLimit,
        MIPFocus=MIPFocus,
        LogToConsole=LogToConsole,
    )

    print('-'*100)
    print(f'Solving model with split type "{split_type.capitalize()}"')
    print('-'*100)

    model.optimize()

    distance_bus, bus_path_df, paths = show_solution(
        model,
        new_stops,
        num_buses,
        new_demand,
        nodes_exceeding_demand,
        distance_matrix_model_pruned,
        stops_df,
    )

    network_plots = plot_networkx_bus(bus_path_df)

    routes_gdf = build_route_df(bus_path_df, depot)

    route_map = plot_routes(
        distance_matrix, bus_path_df, routes_gdf, num_buses, split_type
    )

    return {
        "split_type": split_type,
        "demand": demand,
        "new_demand": new_demand,
        "model": model,
        "distance_bus": distance_bus,
        "bus_path_df": bus_path_df,
        "paths": paths,
        "network_plots": network_plots,
        "routes_gdf": routes_gdf,
        "route_map": route_map,
    }

In [ ]:
NUM_BUSES = 15
BUS_CAPACITY = 80
DEMAND_LIMIT = 120
TIME_LIMIT = 60 * 3
LOG_TO_CONSOLE = 1

## Geometric split

In [ ]:
geometric_split_model = solve_model(
    depot,
    random_stops_df_gpd,
    distance_matrix,
    num_buses=NUM_BUSES,
    BUS_CAPACITY=BUS_CAPACITY,
    DEMAND_LIMIT=DEMAND_LIMIT,
    split_type="geometric",
    TimeLimit=TIME_LIMIT,
    LogToConsole=LOG_TO_CONSOLE,
)

## Capacity split

In [ ]:
capacity_split_model = solve_model(
    depot,
    random_stops_df_gpd,
    distance_matrix,
    num_buses=NUM_BUSES,
    BUS_CAPACITY=BUS_CAPACITY,
    DEMAND_LIMIT=DEMAND_LIMIT,
    split_type="capacity",
    TimeLimit=TIME_LIMIT,
    LogToConsole=LOG_TO_CONSOLE,
)

## Random split

In [ ]:
random_split_model = solve_model(
    depot,  
    random_stops_df_gpd,
    distance_matrix,
    num_buses=NUM_BUSES,
    BUS_CAPACITY=BUS_CAPACITY,
    DEMAND_LIMIT=DEMAND_LIMIT,
    split_type="random",
    TimeLimit=TIME_LIMIT,
    LogToConsole=LOG_TO_CONSOLE,
)

# Compare models

In [ ]:
models = [geometric_split_model, capacity_split_model, random_split_model]

model_bus_routes = []
for model_name, model in zip(['geometric', 'capacity', 'random'], models):
    bus_routes_df = model["bus_path_df"].copy()
    bus_routes_df["MODEL_TYPE"] = model_name
    model_bus_routes.append(bus_routes_df)

model_bus_routes_df = pd.concat(model_bus_routes)

model_bus_routes_df

In [ ]:
# Compare distance travelled, number of buses used, number of split nodes by split_type
model_bus_routes_summary = model_bus_routes_df.groupby("MODEL_TYPE").agg(
    {
        "distance": "sum",
        "bus": "nunique",
        "stop_id": lambda x: len(x.unique()),
        "is_split": "sum",
    }
).rename(
    columns={
        "distance": "Total distance (km)",
        "bus": "Number of buses",
        "stop_id": "Number of stops",
        "is_split": "Number of split nodes",
    }
)

# Add number of 1-stop trips
model_bus_routes_summary = model_bus_routes_summary.merge(
    (
        model_bus_routes_df.groupby(["MODEL_TYPE", "bus"])
        .agg({"step": "max"})
        .query("step == 1")
        .groupby("MODEL_TYPE")
        .size()
        .to_frame()
        .rename(columns={0: "1-stop trips"})
    ),
    left_index=True,
    right_index=True,
)

model_bus_routes_summary.style.background_gradient(
    cmap="GnBu", axis=0
)